In [1]:

#  Setting display options 

import pandas as pd
   #
pd.set_option("display.width", 480)

#  Sets horizontal scroll for wide outputs
#
from IPython.display import display, HTML
display(HTML(""))

from tabulate import tabulate

print("--")


--


#  Setup stuff: Connectivity

In [2]:

from katana import remote
from katana.remote import import_data


my_client = remote.Client()

print(my_client)


In [3]:

NUM_PARTITIONS  = 3
   #
DB_NAME         = "my_db"
GRAPH_NAME      = "my_graph"

print("--")


--


In [16]:

#  DELETE ALL GRAPHS

for l_database in my_client.databases():
   for l_graph in my_client.get_database(name=l_database.name).graphs_in_database():
      l_handle=my_client.get_database(name=l_database.name).get_graph_by_id(id=l_graph.graph_id)
      l_handle.delete()

for l_graph in my_client.graphs():
   print("GRAPH ID: ", l_graph.graph_id, "      GRAPH Version: ", l_graph.version)

print("--")


--


In [4]:

#  DELETE ONE SPECIFIC GRAPH ONLY

for l_database in my_client.databases():
   for l_graph in my_client.get_database(name = l_database.name).graphs_in_database():
      if ((l_database.name == DB_NAME) and (l_graph.name == GRAPH_NAME)):   
         l_handle = my_client.get_database(name = l_database.name).get_graph_by_id(id = l_graph.graph_id)
         l_handle.delete()

for l_graph in my_client.graphs():
   print("GRAPH ID: ", l_graph.graph_id, "      GRAPH Version: ", l_graph.version, "   Graph Name: ", l_graph.name)

print("--")


GRAPH ID:  5e6JRumcZUrTkUNegX9jFeGYpCkMtnrBM9CoQaWuBS5k       GRAPH Version:  3    Graph Name:  my_graph_BB
--


In [ ]:

#  DELETE ALL DATABASES

for l_database in my_client.databases():
   if (l_database.name != "default"):
      my_client.get_database(name=l_database.name).delete_database()
      print("--")

for l_database in my_client.databases():
   print("DB ID: ", l_database.database_id, "     DB Name: ", l_database.name)


In [ ]:

#  CREATE DATABASE

my_database = my_client.create_database(name=DB_NAME)

print(my_database.database_id)


In [5]:

#  CREATE GRAPH
#
my_graph = my_client.get_database(name=DB_NAME).create_graph(name=GRAPH_NAME, num_partitions=NUM_PARTITIONS)

print(my_graph)


<_Graph my_graph, AMrqQvvsEEdXVLM72uaoBZgQBD5abQbFKeG7QsPFAdgY, 0>


In [ ]:

#  CONNECT TO GRAPH

my_graph, *_ = my_client.get_database(name=DB_NAME).find_graphs_by_name(GRAPH_NAME)

print(my_graph)


# Create: Vertices/nodes, edges ..

In [ ]:
# import pandas as pd
# 
# print("--")


In [ ]:

#  Vertices/Nodes, Airports ..

df_airports = pd.DataFrame([
       #
   ["MKE", "Milwaukee"               , "Airport"],
   ["ORD", "Chicago O-Hare"          , "Airport"],
   ["SJC", "San Jose"                , "Airport"],
   ["DEN", "Denver"                  , "Airport"],
       #
   ], columns = ["airport_code", "airport_name", "LABEL"])

print(tabulate(df_airports, headers='keys', tablefmt='psql'))


In [ ]:

#  Create the edge, flights ..

df_flights = pd.DataFrame([
      #
   ["MKE", "ORD",   66, 1, "FLIES_TO" ],
   ["ORD", "MKE",   66, 1, "FLIES_TO" ],
      #
   ["ORD", "DEN",  886, 1, "FLIES_TO" ],
   ["DEN", "ORD",  886, 1, "FLIES_TO" ],
      #
   ["SJC", "DEN",  948, 1, "FLIES_TO" ],                           #  Notice SJC flies to/from Denver
   ["DEN", "SJC",  948, 1, "FLIES_TO" ],
      #
   ["SJC", "ORD", 1829, 1, "FLIES_TO" ],                           #  Notice SJC flies to ORD, but not ORD to SJC
      #
   ], columns = ["START_ID", "END_ID", "DISTANCE", "NUM_HOPS", "TYPE"])

print(tabulate(df_flights, headers='keys', tablefmt='psql'))


In [18]:

# Import the 2 previously created Python DataFrames into KatanaGraph ..

with import_data.DataFrameImporter(my_graph) as df_importer:   
    
   df_importer.nodes_dataframe(df_airports,                    #  Aiport set of Nodes
      id_column             = "airport_code",
      id_space              = "Airport",  
      label                 = "Airport",  
      )
   
   df_importer.edges_dataframe(df_flights,                     #  Our Edge, specifying the relationship between Airport --> FLIES_TO --> Airport
      source_id_space       = "Airport", 
      destination_id_space  = "Airport",   
      source_column         = "START_ID",
      destination_column    = "END_ID",
      type                  = "FLIES_TO"
      )

   df_importer.insert()

print("--")


          0/? [?op/s]

          0/? [?op/s]

--


In [20]:

#  What's in the graph ..

l_result1 = my_graph.query("""
   MATCH ( n ) 
   RETURN n
   """)
print(tabulate(l_result1, headers='keys', tablefmt='psql'))

l_result2 = my_graph.query("""
   MATCH ( n ) - [ r ] -> ( m )
   RETURN r
   """)
print(tabulate(l_result2, headers='keys', tablefmt='psql'))


          0/? [?op/s]

+----+-----------------+-------------+------------------+------------------+----------+-----------+
|    |   n.internal_id | n.labels    | n.airport_code   | n.airport_name   | n.type   | n.LABEL   |
|----+-----------------+-------------+------------------+------------------+----------+-----------|
|  0 |               2 | ['Airport'] | NRT              | Narita, Japan    | node     | nan       |
|  1 |               0 | ['Airport'] | ORD              | Chicago O-Hare   | node     | Airport   |
|  2 |               1 | ['Airport'] | SJC              | San Jose         | node     | Airport   |
|  3 | 281474976710656 | ['Airport'] | ICN              | Seoul, Korea     | node     | nan       |
|  4 | 562949953421313 | ['Airport'] | DEN              | Denver           | node     | Airport   |
|  5 | 562949953421312 | ['Airport'] | MKE              | Milwaukee        | node     | Airport   |
+----+-----------------+-------------+------------------+------------------+----------+-----------+


          0/? [?op/s]

+----+-----------------+----------------+-----------------+-----------+--------------+--------------+----------+-----------------+------------------+----------+
|    |        r.end.id | r.end.labels   |   r.internal_id | r.label   |   r.DISTANCE |   r.NUM_HOPS | r.TYPE   |      r.start.id | r.start.labels   | r.type   |
|----+-----------------+----------------+-----------------+-----------+--------------+--------------+----------+-----------------+------------------+----------|
|  0 | 562949953421313 | ['Airport']    |               1 | FLIES_TO  |          886 |            1 | FLIES_TO |               0 | ['Airport']      | edge     |
|  1 | 562949953421312 | ['Airport']    |               3 | FLIES_TO  |           66 |            1 | FLIES_TO |               0 | ['Airport']      | edge     |
|  2 |               0 | ['Airport']    |               0 | FLIES_TO  |         1829 |            1 | FLIES_TO |               1 | ['Airport']      | edge     |
|  3 | 562949953421313 | ['Airport

In [ ]:

#  What's in the edge .. 

l_result = my_graph.query("""

   MATCH ( n )  - [r] ->  (m)
   RETURN ID(n) AS FROM, ID(m) AS TO, r.DISTANCE AS DISTANCE, r.NUM_HOPS AS NUM_HOPS, TYPE(r) AS my_type, LABELS(n) AS FROM_LABELS, LABELS(m) AS TO_LABELS
   
   """)

print(tabulate(l_result, headers='keys', tablefmt='psql'))
